In [1]:
# Work on reimplementing Andrej Karpathy's nano-gpt from this video
# https://youtu.be/kCc8FmEb1nY?si=cdnNJrPwDEPQPn0C

In [150]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [3]:
# load data
with open("input.txt", mode='r') as f:
  words = f.read()

In [4]:
print(len(words))
print(words[:1000])

1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for re

In [6]:
# vocab
vocab = sorted(set(words))
vocab_size = len(vocab)
print(vocab_size)
print(''.join(vocab))

65

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [96]:
# simple character level tokenization
stoi = {c:i for i,c in enumerate(vocab)}
itos = {i:c for i,c in enumerate(vocab)}
# encode and decode functions
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])
print(encode('Is it working?'))
decode(encode('Is it working?'))

[21, 57, 1, 47, 58, 1, 61, 53, 56, 49, 47, 52, 45, 12]


'Is it working?'

In [97]:
# we'd like to use torch tensors for training NNs
data = torch.tensor(encode(words), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [98]:
# train/validation split
n = int(0.9 * len(words))
train = data[:n]
test = data[n:]
print(f'train size: {len(train)}, test size: {len(test)}')

train size: 1003854, test size: 111540


In [10]:
# context size = one sampled block gives us block_size of examples to train on
block_size = 8
block = train[:block_size+1]
print(block)
for i in range(1, block_size+1):
  print(f'for input context: {block[:i]} target is: {block[i]}')

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])
for input context: tensor([18]) target is: 47
for input context: tensor([18, 47]) target is: 56
for input context: tensor([18, 47, 56]) target is: 57
for input context: tensor([18, 47, 56, 57]) target is: 58
for input context: tensor([18, 47, 56, 57, 58]) target is: 1
for input context: tensor([18, 47, 56, 57, 58,  1]) target is: 15
for input context: tensor([18, 47, 56, 57, 58,  1, 15]) target is: 47
for input context: tensor([18, 47, 56, 57, 58,  1, 15, 47]) target is: 58


In [11]:
# so training examples are defined like this
x = train[:block_size]
y = train[1:block_size+1]
print(x)
print(y)

tensor([18, 47, 56, 57, 58,  1, 15, 47])
tensor([47, 56, 57, 58,  1, 15, 47, 58])


In [99]:
# we want to work with batches of data as we aren't going to feed in the whole dataset
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
  data = train if split == "train" else test
  ix = torch.randint(len(data) - block_size, (batch_size,)) # randint range is exclusive
  x, y = [], []
  for i in ix:
    x.append(data[i:i+block_size])
    y.append(data[i+1:i+block_size+1])
  x = torch.stack(x)
  y = torch.stack(y)
  return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)
print('---------')

for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b, t]
    print(f'for input context: {context.tolist()} target is: {target}')

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
---------
for input context: [24] target is: 43
for input context: [24, 43] target is: 58
for input context: [24, 43, 58] target is: 5
for input context: [24, 43, 58, 5] target is: 57
for input context: [24, 43, 58, 5, 57] target is: 1
for input context: [24, 43, 58, 5, 57, 1] target is: 46
for input context: [24, 43, 58, 5, 57, 1, 46] target is: 43
for input context: [24, 43, 58, 5, 57, 1, 46, 43] target is: 39
for input context: [44] target is: 53
for input context: [44, 53] target is: 56
for input context: [44, 53, 56] target is: 1
for input context: [44, 53, 56, 1] target is: 58
for input context

In [101]:
# bigram character-level language mode
torch.manual_seed(1337)
batch_size = 32

class BigramModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_lookup_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, x, target=None):
    logits = self.token_lookup_table(x)
    if target is None:
      loss = None
    else:
      B, T, C = logits.shape
      # to conform to pytorch cross-entropy signature
      logits = logits.view(B*T, C)
      target = target.view(B*T)
      loss = F.cross_entropy(logits, target)
    return logits, loss

  @torch.no_grad()
  def generate(self, idx, max_num_tokens):
    # idx is (B, T) array of indices in the current context
    for _ in range(max_num_tokens):
      logits, loss = self(idx)
      # only last time step
      logits = logits[:, -1, :] # (B, C)
      probs = F.softmax(logits, dim=-1) # (B, C)
      next_i = torch.multinomial(probs, num_samples=1) # (B, 1)
      idx = torch.cat((idx, next_i), dim=1) # (B, T+1)
    return idx


model = BigramModel(vocab_size)
Xb, Yb = get_batch('train')
logits, loss = model(Xb, Yb)
print(loss.item())

gen_tokens = model.generate(torch.zeros((1, 1), dtype=torch.long), 100)
print(decode(gen_tokens[0].tolist()))

4.648484230041504

p fvLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3!dcbf?pGXepydZJSrF$Jrqt!:wwWSzPN


In [102]:
# use adam optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

In [111]:
# training loop
for i in range(10000):
  Xb, Yb = get_batch('train')           # get random batch of training examples
  logits, loss = model(Xb, Yb)          # do forward pass and compute loss
  optimizer.zero_grad(set_to_none=True) # set grads of model parameters to None
  loss.backward()                       # backward pass, compute grads
  optimizer.step()                      # update model parameters
print(loss.item())

2.4569950103759766


In [112]:
gen_tokens = model.generate(torch.zeros((1, 1), dtype=torch.long), 500)
print(decode(gen_tokens[0].tolist()))


S: l, whikstter melceat it cr the, f peditesushistou YOfed
Wifer t we of I;
PStorithe avend Whatin ct are h,
con f!
O, obleleapt ailk nd wit, nd alo willext sel nesater nco hechawllionoril?
Banou t ve ushihth; IORof ithepailacais h t ast s
t Is t be IOME ithutarin!

Thend l msk ROLETr t y wie it hisel h mernth ictheng c,
ABEMangeilogerquzen t s gng soued thay fre e wh:
Tond tinyowasagZetlar t gulan, ns.
HAnsthend rdl m tomatedereren prad?
m, there band atoy ma GLO:
Tald.
IIUS:

Maver plon us aye


## Math trick to understand self-attention

In [27]:
# Matrix multiply with (T, T) matrix gives us a way to aggregate information in more efficient way
# Here just summing up columns
a = torch.ones((T, T))
b = torch.randint_like(x, 4)
c = a @ b

print('a', a)
print('-----')
print('b',  b)
print('-----')
print('c', c)

a tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])
-----
b tensor([[[0., 0., 0.],
         [1., 2., 0.],
         [0., 1., 2.],
         [3., 0., 0.]],

        [[2., 1., 3.],
         [2., 1., 1.],
         [1., 1., 2.],
         [2., 0., 1.]]])
-----
c tensor([[[4., 3., 2.],
         [4., 3., 2.],
         [4., 3., 2.],
         [4., 3., 2.]],

        [[7., 3., 7.],
         [7., 3., 7.],
         [7., 3., 7.],
         [7., 3., 7.]]])


In [29]:
# but tril gives it extra power to insist on not looking ahead
# and now we have a rolling sum of columns in tensor b (in every batch, as we would like to)
a = torch.tril(torch.ones((T,T)))
b = torch.randint_like(x, 4)
c = a @ b

print('a', a)
print('-----')
print('b',  b)
print('-----')
print('c', c)

a tensor([[1., 0., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 1.]])
-----
b tensor([[[2., 3., 0.],
         [2., 1., 2.],
         [1., 1., 1.],
         [3., 3., 1.]],

        [[3., 2., 2.],
         [2., 0., 1.],
         [1., 0., 2.],
         [0., 3., 1.]]])
-----
c tensor([[[2., 3., 0.],
         [4., 4., 2.],
         [5., 5., 3.],
         [8., 8., 4.]],

        [[3., 2., 2.],
         [5., 2., 3.],
         [6., 2., 5.],
         [6., 5., 6.]]])


In [33]:
# normalize a, such that rows sum to one and we have rolling average
a = torch.tril(torch.ones((T,T)))
a = a / a.sum(1, keepdim=True)
b = torch.randint_like(x, 4)
c = a @ b

print('a', a)
print('-----')
print('b',  b)
print('-----')
print('c', c)

a tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500]])
-----
b tensor([[[0., 3., 1.],
         [1., 3., 1.],
         [0., 0., 0.],
         [3., 3., 1.]],

        [[3., 3., 0.],
         [1., 3., 2.],
         [2., 1., 1.],
         [3., 0., 2.]]])
-----
c tensor([[[0.0000, 3.0000, 1.0000],
         [0.5000, 3.0000, 1.0000],
         [0.3333, 2.0000, 0.6667],
         [1.0000, 2.2500, 0.7500]],

        [[3.0000, 3.0000, 0.0000],
         [2.0000, 3.0000, 1.0000],
         [2.0000, 2.3333, 1.0000],
         [2.2500, 1.7500, 1.2500]]])


In [55]:
# in bigram-language model we only consider one token to predict next
# to go one step further in utilizing context, we can consider an average of
# all previous tokens in the block (version 1)
B, T, C = 2, 4, 3 # shape
x = torch.randint(4, (B,T,C)) * 1.0
xbow = torch.zeros_like(x)
for b in range(B):
  for t in range(T):
    xbow[b, t] = x[b, :t+1].mean(dim=0)

In [56]:
x

tensor([[[3., 1., 2.],
         [0., 2., 1.],
         [1., 0., 0.],
         [3., 0., 0.]],

        [[0., 0., 2.],
         [2., 0., 3.],
         [3., 0., 3.],
         [3., 1., 1.]]])

In [57]:
xbow

tensor([[[3.0000, 1.0000, 2.0000],
         [1.5000, 1.5000, 1.5000],
         [1.3333, 1.0000, 1.0000],
         [1.7500, 0.7500, 0.7500]],

        [[0.0000, 0.0000, 2.0000],
         [1.0000, 0.0000, 2.5000],
         [1.6667, 0.0000, 2.6667],
         [2.0000, 0.2500, 2.2500]]])

In [58]:
# Now we have created xbow (bag of words), which has agregated information about all previous time steps + self via mean. 

In [59]:
# normalize tril to get rolling average via matmul (version 2)
wei = torch.tril(torch.ones((T, T)))
wei /= wei.sum(1, keepdim=True)
print('wei', wei)
xbow2 = wei @ x
torch.allclose(xbow, xbow2)


wei tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500]])


True

In [60]:
# normalize tril via softmax (version 3)
# not for this specific case, but I guess if we use normalization as in version 2
# and get unlucky with sum to be very close to 0, we can blow up the tensor
# softmax avoids it, usig - max(tensor) so largest number is 0 and exp goes to max 1
mask = torch.tril(torch.ones((T, T)))
wei = torch.masked_fill(torch.zeros_like(wei), mask==0, -torch.inf)
wei = F.softmax(wei, 1)
print('wei', wei)
xbow2 = wei @ x
torch.allclose(xbow, xbow2)

wei tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500]])


True

In [139]:
# move to scaled dotproduct self-attention (version 4)
torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn((B, T, C))

head_size = 16

key = nn.Linear(C, head_size, bias=False) # (C, 16)
nn.init.xavier_uniform_(key.weight)
query = nn.Linear(C, head_size, bias=False) # (C, 16)
nn.init.xavier_uniform_(query.weight)
value = nn.Linear(C, head_size, bias=False) # (C, 16)
nn.init.xavier_uniform_(value.weight)

# project into communication subspace
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)

# k (B, T, 16) @  q (B, T, 16) [need transpose] --> (B, T, T)
wei = q @ k.transpose(-1, -2) # communication in search for affininty between tokens
wei = wei * (head_size **-0.5) # scale to preserve var, we don't want softmax downstream
# to bo too peaky

tril = torch.tril(torch.ones((T,T)))
# wei = torch.zeros((T,T))
wei = torch.masked_fill(wei, tril==0, -torch.inf) # mask forbiden communication
wei = F.softmax(wei, dim=-1) # make found affinities to become more like weights

# projection of my "values"
v = value(x) # (B, T, 16)
# now we "weight" our "values" with previously found affinities
out = wei @ v # (B, T, 16)
out.shape

torch.Size([4, 8, 16])

In [140]:
# default kaming init of linear layer
# comment weight init lines in previous cell to see the change
key.weight.var()

tensor(0.0403, grad_fn=<VarBackward0>)

In [141]:
x.var(), k.var(), q.var()

(tensor(1.0731),
 tensor(1.3544, grad_fn=<VarBackward0>),
 tensor(1.4393, grad_fn=<VarBackward0>))

In [142]:
(q @ k.transpose(-1, -2)).var()

tensor(37.1898, grad_fn=<VarBackward0>)

In [143]:
((q @ k.transpose(-1, -2))* (head_size **-0.5)).var()

tensor(2.3244, grad_fn=<VarBackward0>)

In [144]:
wei.var()

tensor(0.0499, grad_fn=<VarBackward0>)

In [145]:
out.var()

tensor(0.8124, grad_fn=<VarBackward0>)

## Explore Init

In [148]:
# Here I want to elaborate and explore how different initialization schemas
# may influence GPT model training. As we know, given matrices A and B, variance
# of A @ B depends multiplicatively on var(A), var(B) and their common shape size
# (A.shape[1] == B.shape[0]). As we do a lot of matrix multiplications (and we do
# a loooot in GPT), the deeper we go the more variance will increase or decrease
# leading to vanishing or exploding activations and eventually gradients, preventing or
# slowing down learning. LayerNorm and residual connections employed to help with this issue,
# one by renormalizing activations and another by allowing signal to propagate through the layer
# unaffected by the layer computation and effectively give a way to gradient flow through these
# layers unaffected. The natural question is: does initialization really matter?
# GPT 1 paper says, that given extensive use of LayerNorm we simply initialize weights with N(0, 0.02)
# But if LayerNorm and residual connections would solve all the problems, why bother with specific init strategy?
# My assumption is, that it does matter and along with LayerNorm and residual connections allow to train 
# very deep and wide transformers and achieve better loss minima.

# Might be actually because of attention softmaxes become more and more peaky with activation var (and some values) grow to large.

In [149]:
# So, to answer the question or at least to find a hint, I would like to compare different initializations:
# 1. Default (pytorch linear: kamin uniform, embedding: N(0, 1)); 2. One I found tinkering with attention
# in the previous section (xavier uniform); 3. GPT 1 paper N(0, 0.02) for all linear layers and embeddings.

In [163]:
# Hyperparameters
batch_size = 64
block_size = 256
device = "cuda" if torch.cuda.is_available() else "cpu"
learining_rate = 3e-4
n_embd = 384
n_heads = 6
n_layers = 6
dropout = 0.0

In [164]:
# Load and prepare the data
with open("input.txt", mode="r") as f:
    words = f.read()

# Vocab and tokenizer
vocab = sorted(set(words))
vocab_size = len(vocab)
# mappings
stoi = {c: i for i, c in enumerate(vocab)}
itos = {i: c for i, c in enumerate(vocab)}
# encoder and decoder
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])
# to tensor and train/test split
data = torch.tensor(encode(words), dtype=torch.long)
n = int(0.9 * len(data))
train = data[:n]
test = data[n:]


In [194]:
# Attention
class SelfAttention(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.head_size = head_size
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.do = nn.Dropout(dropout)
        self.register_buffer("tril", torch.tril(torch.ones((block_size, block_size))))

    def forward(self, x):
        B, T, C = x.shape
        q = self.query(x)  # (B, T, H)
        k = self.key(x)  # (B, T, H)
        aff = q @ k.transpose(-1, -2)  # (B, T, T)
        aff *= self.head_size**-0.5  # scale to preserve var
        # prevent communication with future tokens
        aff = torch.masked_fill(aff, self.tril[:T, :T] == 0, -torch.inf)
        aff = F.softmax(aff, dim=-1)
        aff = self.do(aff)
        # value
        v = self.value(x)  # (B, T, H)
        out = aff @ v  # (B, T, H)
        return out


# Multihead Attention
class MultiHeadAttention(nn.Module):
    def __init__(self, n_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([SelfAttention(head_size) for _ in range(n_heads)])
        self.proj = nn.Linear(
            n_heads * head_size, n_embd
        )  # in case we need to project back into C
        self.do = nn.Dropout(dropout)

    def forward(self, x):
        # x (B, T, C) and H = C / n_heads
        x = torch.concat([head(x) for head in self.heads], dim=-1)  # (B,T,C)
        out = self.do(self.proj(x))
        return out


# Feed Forward
class FeedForward(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, n_embd * 4),
            nn.ReLU(),
            nn.Linear(n_embd * 4, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        # x (B,T,C)
        out = self.net(x)
        return out


# Block - multi-head attention followed by feed-forward
class Block(nn.Module):
    def __init__(self, n_heads, head_size):
        super().__init__()
        self.mha = MultiHeadAttention(n_heads, head_size)
        self.ff = FeedForward()
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        # x (B,T,C)
        # pre-norm
        # x = x + self.mha(self.ln1(x))  # (B,T,C)
        # x = x + self.ff(self.ln2(x))  # (B,T,C)
        # post-norm
        x = self.ln1(x + self.mha(x))
        x = self.ln2(x + self.ff(x))
        return x


# Model definition
class GPTLanguageModel(nn.Module):
    def __init__(self, init_fn):
        super().__init__()
        self.token_embed_table = nn.Embedding(vocab_size, n_embd)
        self.position_embed_table = nn.Embedding(block_size, n_embd)
        self.emb_do = nn.Dropout(dropout)
        self.blocks = nn.ModuleList(
            [Block(n_heads, n_embd // n_heads) for _ in range(n_layers)]
        )
        self.ln = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.apply(init_fn)

    def forward(self, x, targets=None):
        B, T = x.shape
        tok_embeds = self.token_embed_table(x)  # (B, T, C)
        pos_embeds = self.position_embed_table(torch.arange(T, device=device))  # (T, C)
        x = tok_embeds + pos_embeds  # (B, T, C)
        x = self.emb_do(x)
        for block in self.blocks:
            x = block(x)  # (B, T, C)
        x = self.ln(x)
        logits = self.lm_head(x)  # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss


In [195]:
# get batch of random examples
def get_batch(split):
  data = train if split == "train" else test
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i : i + block_size] for i in ix])
  y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
  x, y = x.to(device), y.to(device)
  return x, y

In [196]:
def init_default(m):
  pass

def init_xavier_uniform(m):
  if isinstance(m, nn.Linear):
    nn.init.xavier_uniform_(m.weight)
    if m.bias is not None:
      nn.init.zeros_(m.bias)
  if isinstance(m, nn.Embedding):
    nn.init.xavier_uniform_(m.weight)

def init_normal(m):
  if isinstance(m, nn.Linear):
    nn.init.normal_(m.weight, 0.0, 0.02)
    if m.bias is not None:
      nn.init.zeros_(m.bias)
  if isinstance(m, nn.Embedding):
    nn.init.normal_(m.weight, 0.0, 0.02)

In [212]:
# Initialize model and optimizer
model = GPTLanguageModel(init_normal)
model.train()
model.to(device)
# remove weight decay for now
optimizer = torch.optim.AdamW(model.parameters(), lr=learining_rate, weight_decay=0.0)

In [213]:
torch.manual_seed(1337)
Xb, yb = get_batch('train')

In [214]:
for i in range(100):
  logits, loss = model(Xb, yb)
  if i == 0:
    print('logits var at iter 1:', logits.var())
    print('loss', loss.item())
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()
print('logits var after 100 iterations:', logits.var())
print('loss', loss.item())

logits var at iter 1: tensor(0.1509, device='cuda:0', grad_fn=<VarBackward0>)
loss 4.241641998291016
logits var after 100 iterations: tensor(3.0261, device='cuda:0', grad_fn=<VarBackward0>)
loss 1.3023432493209839


In [193]:
# 1. Default pytorch initialization: var(logits) @1st iteration = 0.32; @100th iteration = 3.47
# 2. Xavier uniform initialization: @1st iteration = 1.72; @100th iteration = 6.5
# 3. N(0, 0.02) initialization: @1st iteration = 0.135; @100th iteration = 2.76

# This experiment shows, that N(0,0.02) init shows the smallest absolute increase of logits variance,
# potentially allowing network to train longer w/o activations growth to fast and slowing down training
# in the later stages (through softmax becoming too peaky and/or overall zeroing down too many gradients
# too soon).